In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [47]:
stfold = StratifiedKFold(n_splits=5, shuffle=True)
train_l = list()
val_l = list()
test_l = list()
maxlen = 100
cnt = 0
for train_index, test_index in stfold.split(data['comment'], data['label']):
    cnt += 1
    train = data.iloc[train_index]
    train_x, train_y = train['comment'], train['label']
    tmp = data.iloc[test_index]

    val, test = train_test_split(tmp, test_size=0.5, stratify=tmp['label'])    
    val_x, val_y = val['comment'], val['label']
    test_x, test_y = test['comment'], test['label']    
    
    train_x = keras.preprocessing.sequence.pad_sequences(train_x, maxlen=maxlen)
    val_x = keras.preprocessing.sequence.pad_sequences(val_x, maxlen=maxlen)
    test_x = keras.preprocessing.sequence.pad_sequences(test_x, maxlen=maxlen)
    
    embed_dim = 128  # Embedding size for each token
    vocab_size = n_word_unique + 1

    inputs = layers.Input(shape=(maxlen, ))
    embedding_layer = layers.Embedding(input_dim = vocab_size, output_dim = embed_dim, input_length=maxlen)
    x = embedding_layer(inputs)
    x = tf.keras.layers.Flatten()(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model_path = "sigmoid_models/{itr:04}/".format(itr=cnt)
    checkpoint_path = model_path + "{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    !mkdir sigmoid_models
    !mkdir $model_path
    
    
    model.compile(tf.keras.optimizers.Adam(lr=0.001), "binary_crossentropy", metrics=["accuracy", 
                                                          tf.keras.metrics.AUC(name='auc')])
    
    

    model.save_weights(checkpoint_path.format(epoch=0))

    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1,
            save_weights_only=True, save_best_only=True, mode='auto')
    
    history = model.fit(
                        train_x, train_y, 
                        batch_size=128, epochs=10, 
                        validation_data=(val_x, val_y),
                        callbacks = [checkpoint]
                        )

    latest = tf.train.latest_checkpoint(checkpoint_dir)
    model.load_weights(latest)
    
    train_eval = model.evaluate(train_x, train_y)
    val_eval = model.evaluate(val_x, val_y)
    test_eval = model.evaluate(test_x, test_y)
    
    test_l.append(test_eval)
    val_l.append(val_eval)
    train_l.append(train_eval)

Epoch 1/10
62/63 [============================>.] - ETA: 0s - loss: 0.6786 - accuracy: 0.5615 - auc: 0.5873
Epoch 00001: val_loss improved from inf to 0.66481, saving model to sigmoid_models/0001/0001.ckpt
63/63 [==============================] - 3s 40ms/step - loss: 0.6785 - accuracy: 0.5620 - auc: 0.5871 - val_loss: 0.6648 - val_accuracy: 0.5790 - val_auc: 0.6466
Epoch 2/10
63/63 [==============================] - ETA: 0s - loss: 0.5569 - accuracy: 0.7935 - auc: 0.8631
Epoch 00002: val_loss improved from 0.66481 to 0.57433, saving model to sigmoid_models/0001/0002.ckpt
63/63 [==============================] - 2s 35ms/step - loss: 0.5569 - accuracy: 0.7935 - auc: 0.8631 - val_loss: 0.5743 - val_accuracy: 0.7030 - val_auc: 0.7954
Epoch 3/10
63/63 [==============================] - ETA: 0s - loss: 0.3056 - accuracy: 0.9499 - auc: 0.9917
Epoch 00003: val_loss improved from 0.57433 to 0.48393, saving model to sigmoid_models/0001/0003.ckpt
63/63 [==============================] - 2s 32ms/s

In [48]:
test_l = np.array(test_l)
val_l = np.array(val_l)
train_l = np.array(train_l)

print("test avg loss: ", np.mean(test_l[:, 0]), "+/-" ,np.std(test_l[:, 0]))
print("test avg acc: ", np.mean(test_l[:, 1]),  "+/-" ,np.std(test_l[:, 1]))
print("test avg auc: ", np.mean(test_l[:, 2]),  "+/-" ,np.std(test_l[:, 2]))

print('\n')


print("val avg loss: ", np.mean(val_l[:, 0]), "+/-" ,np.std(val_l[:, 0]))
print("val avg acc: ", np.mean(val_l[:, 1]),  "+/-" ,np.std(val_l[:, 1]))
print("val avg auc: ", np.mean(val_l[:, 2]),  "+/-" ,np.std(val_l[:, 2]))


print('\n')


print("train avg loss: ", np.mean(train_l[:, 0]), "+/-" ,np.std(train_l[:, 0]))
print("train avg acc: ", np.mean(train_l[:, 1]),  "+/-" ,np.std(train_l[:, 1]))
print("train avg auc: ", np.mean(train_l[:, 2]),  "+/-" ,np.std(train_l[:, 2]))

test avg loss:  0.44516376256942747 +/- 0.018611459252890164
test avg acc:  0.7971999883651734 +/- 0.01248037389166063
test avg auc:  0.8711098194122314 +/- 0.012005653474350361


val avg loss:  0.43475946187973025 +/- 0.015502572498735774
val avg acc:  0.8044000029563904 +/- 0.008754429669245903
val avg auc:  0.8762329459190369 +/- 0.010071336664155136


train avg loss:  0.019328803569078446 +/- 0.005075406664754881
train avg acc:  0.9994250059127807 +/- 0.0003316662962928054
train avg auc:  0.9999982714653015 +/- 1.9039929195897843e-06
